# Preprocessing

In [6]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion formant bandwith

In [7]:
import numpy as np
from scipy.signal import lfilter
from scipy.linalg import toeplitz

def calculate_formant_bandwidths(audio_signal, sample_rate=16000, lpc_order=12, num_formants=3):
    """
    Berechnet die Bandbreiten der ersten Formantfrequenzen eines Audiosignals mit LPC-Analyse.

    Quelle:
        Markel, J. D., & Gray, A. H. (1976). Linear Prediction of Speech.
        Springer-Verlag.

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).
        lpc_order (int): Ordnung des LPC-Modells (Standard: 12).
        num_formants (int): Anzahl der zu extrahierenden Formanten (Standard: 3).

    Returns:
        np.array: Die geschätzten Formant-Bandbreiten (in Hz), ohne negative Werte.
    """
    # Berechnung der Autokorrelation
    autocorr = np.correlate(audio_signal, audio_signal, mode='full')
    autocorr = autocorr[len(autocorr) // 2:]  # Nur positive Hälfte verwenden

    # Yule-Walker-Gleichung lösen, um LPC-Koeffizienten zu berechnen
    R = toeplitz(autocorr[:lpc_order])
    r = autocorr[1:lpc_order+1]
    
    try:
        lpc_coeffs = np.linalg.solve(R, r)
    except np.linalg.LinAlgError:
        return np.array([])  # Falls Matrix singulär ist (z. B. Stille), keine Bandbreiten zurückgeben

    # Berechnung der Polstellen des LPC-Filters
    roots = np.roots(np.concatenate([[1], -lpc_coeffs]))  # LPC-Koeffizienten als Filter nutzen
    roots = [r for r in roots if np.imag(r) >= 0]  # Nur positive Frequenzen behalten

    # Umwandlung in Frequenzen & Bandbreiten
    formant_frequencies = np.angle(roots) * (sample_rate / (2 * np.pi))
    formant_bandwidths = -np.log(np.abs(roots)) * (sample_rate / (2 * np.pi))

    # Entferne unrealistische Werte (< 50 Hz) und sortiere nach Frequenz
    valid_indices = np.where(formant_frequencies > 50)[0]
    formant_frequencies = formant_frequencies[valid_indices]
    formant_bandwidths = np.abs(formant_bandwidths[valid_indices])  # 🚀 **Jetzt absolute Werte verwenden!**

    # Sortieren nach Frequenz und die ersten num_formants Formanten zurückgeben
    sorted_indices = np.argsort(formant_frequencies)
    formant_frequencies = formant_frequencies[sorted_indices][:num_formants]
    formant_bandwidths = formant_bandwidths[sorted_indices][:num_formants]

    return formant_bandwidths

# Test

In [8]:
import os

# Formant-Bandbreiten für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)

        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)

        # Formant-Bandbreiten berechnen
        formant_bw_values = calculate_formant_bandwidths(audio_signal)

        print(f"Formant Bandwidths ({file_name}): {formant_bw_values}")

Formant Bandwidths (_noise_pink.wav): [ 530.30780398  979.64439672 1173.87802507]
Formant Bandwidths (_noise_white.wav): [1127.59281052 1271.38337724 1120.16125603]
Formant Bandwidths (_signal_constant.wav): [2320.77741829 2383.13316393 2430.06913224]
Formant Bandwidths (_signal_silence.wav): []
Formant Bandwidths (_signal_sine.wav): [1.61607138e-01 8.18445310e+02 7.79045494e+02]
Formant Bandwidths (_snr_03.wav): [13.7321026  25.0008993  15.94748512]
Formant Bandwidths (_snr_10.wav): [48.81737391 88.86442346 68.44255446]
Formant Bandwidths (_snr_20.wav): [19.67952523 38.2495487  31.35680243]
Formant Bandwidths (example1.wav): [ 42.63445569 724.8428424  891.51458983]
Formant Bandwidths (example2.wav): [ 28.10326137 644.63529641 697.93254818]
Formant Bandwidths (example3.wav): [ 89.92243296 748.00195127 892.32927978]
